In [46]:
#Get Trump tweets by time he was spending doing things

#"https://docs.google.com/spreadsheets/d/1oITCuVsYdhNXtY7GElLelsrbjRRIPJ1ce-_v-8J1X_A/edit#gid=0"


import tweepy
import pandas as pd
import numpy as np
from datetime import timezone, datetime, timedelta
import credentials
import qgrid
from IPython.display import display, HTML

In [47]:

auth = tweepy.OAuthHandler(credentials.consumer_key_contest, credentials.consumer_secret_contest)
auth.set_access_token(credentials.access_token_contest, credentials.access_token_secret_contest)
api = tweepy.API(auth)

account = "realDonaldTrump"


private_schedule = pd.read_csv('Axios _ President Donald Trump Private Schedules, Nov. 7, 2018 to Feb. 2, 2019 - data.csv')
    
twitter_deets = tweepy.Cursor(api.user_timeline, id=account, tweet_mode='extended').items(1000)
#Create start and end time columns
private_schedule['start_date'] = pd.to_datetime(private_schedule['date'] + ' ' + private_schedule['time_start'])
private_schedule['end_date'] = pd.to_datetime(private_schedule['date'] + ' ' + private_schedule['time_end'])
private_schedule = private_schedule.drop(columns=['date', 'time_start','time_end'])
#private_schedule['unassigned_time'] = np.where((private_schedule['end_date'] not in private_schedule['start_date']),True,False )

all_tweets = pd.DataFrame()
for i in twitter_deets:
    temp=pd.DataFrame()
    temp['tweet_timestamp'] = [i.created_at]
    temp['tweet_text'] = [i.full_text]
    temp['from_source'] = [i.source]
    all_tweets = all_tweets.append(temp)

#Convert date variable to datetime
all_tweets['tweet_timestamp'] = (all_tweets['tweet_timestamp']
                                    .astype(str)
                                    .apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S')))
#Subtract five hours to adjust for timezone
all_tweets['tweet_timestamp'] = all_tweets['tweet_timestamp'].apply(lambda x: x - timedelta(hours=5))



#Allocate tweet to appropriate time
private_schedule= private_schedule.assign(keys=1)
all_tweets = all_tweets.assign(keys=1)

tweets_and_schedule = (pd.merge(private_schedule, all_tweets, on = 'keys', how='outer')
                .query('start_date < tweet_timestamp and end_date> tweet_timestamp')
                .reset_index())
#tweets_and_schedule[tweets_and_schedule['top_category']=='executive_time']['tweet_text']
tweets_and_schedule = tweets_and_schedule.drop(columns=['keys'])

#tweets_and_schedule[['tweet_text','top_category']].head()
#Save database
tweets_and_schedule.to_csv('tweets and schedule.csv', index=False)
#timezone.fromutc(all_tweets['tweet_timestamp'].iloc[0])
#datetime.fromtimestamp(temp['tweet_timestamp'].iloc[0])




In [48]:
trump_tweets = pd.read_csv('tweets and schedule.csv')

In [49]:
display(HTML(trump_tweets.to_html()))

,index,week,duration,listed_title,top_category,listed_location,listed_project_officer,detail_category,notes,start_date,end_date,tweet_timestamp,tweet_text,from_source


In [50]:
tweet_table = qgrid.show_grid(trump_tweets, show_toolbar=True)
tweet_table

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [42]:
tweet_table.get_changed_df()

,index,week,duration,listed_title,top_category,listed_location,listed_project_officer,detail_category,notes,start_date,end_date,tweet_timestamp,tweet_text,from_source
346,572014,13,0.500000,Media engagement,event,Diplomatic Room,Sarah Sanders,media_engagement,NaN,2019-02-01 15:00,2019-02-01 15:30,2019-02-01 15:07,National African American History Month is an ...,Twitter for iPhone
71,115709,4,1.250000,Photos at United States secret service christm...,event,Diplomatic reception room,Lindsay Reynolds,Photo_opportunity,NaN,2018-11-27 19:30,2018-11-27 20:45,2018-11-27 19:31,The Mueller Witch Hunt is a total disgrace. Th...,Twitter for iPhone
96,161670,4,1.750000,G20 Leaders' dinner,event,"Buenos Aires, Argentina",Ambassador John Bolton,g20_dinner,NaN,2018-11-30 18:15,2018-11-30 20:00,2018-11-30 19:23,"Great day at the #G20Summit in Buenos Aires, A...",Twitter for iPhone
162,286527,7,0.500000,Media engagement,event,Oval Office,Sarah Sanders,media_engagement,NaN,2018-12-17 16:00,2018-12-17 16:30,2018-12-17 16:14,Today I am making good on my promise to defend...,Twitter for iPhone
156,274550,6,0.250000,Remarks at christmas reception,event,Grand foyer,Lindsay Reynolds,christmas_remarks,NaN,2018-12-14 17:15,2018-12-14 17:30,2018-12-14 17:18,....I look forward to working with him in this...,Twitter for iPhone
327,547042,13,0.500000,Media engagement,event,Oval office,Sarah Sanders,media_engagement,NaN,2019-01-30 16:30,2019-01-30 17:00,2019-01-30 16:58,....Large protests all across Venezuela today ...,Twitter for iPhone
328,547043,13,0.500000,Media engagement,event,Oval office,Sarah Sanders,media_engagement,NaN,2019-01-30 16:30,2019-01-30 17:00,2019-01-30 16:58,Spoke today with Venezuelan Interim President ...,Twitter for iPhone
329,547044,13,0.500000,Media engagement,event,Oval office,Sarah Sanders,media_engagement,NaN,2019-01-30 16:30,2019-01-30 17:00,2019-01-30 16:54,"Dow just broke 25,000. Tremendous news!",Twitter for iPhone
81,119697,4,0.500000,Media engagement,event,Oval Office,Sarah Sanders,media_engagement,NaN,2018-11-28 10:45,2018-11-28 11:15,2018-11-28 11:09,Steel Dynamics announced that it will build a ...,Twitter for iPhone
157,274551,6,0.250000,Remarks at christmas reception,event,Grand foyer,Lindsay Reynolds,christmas_remarks,NaN,2018-12-14 17:15,2018-12-14 17:30,2018-12-14 17:18,"I am pleased to announce that Mick Mulvaney, D...",Twitter for iPhone
